In [5]:
#Webcam Heartbeat Detector
#Devin Holz, Troy Cunningham, Cada Kato

import numpy as np
import cv2
import sys

#initialize webcam capture
cap = cv2.VideoCapture(0)

# Set video size to 640 by 480
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
print("Video size: ", frame_width, "x", frame_height)

# Helper Methods
def buildGauss(frame, levels):
    pyramid = [frame]
    for level in range(levels):
        frame = cv2.pyrDown(frame)
        pyramid.append(frame)
    return pyramid
    
def reconstructFrame(pyramid, index, levels):
    filteredFrame = pyramid[index]
    for level in range(levels):
        filteredFrame = cv2.pyrUp(filteredFrame)
    filteredFrame = filteredFrame[:videoHeight, :videoWidth]
    return filteredFrame

def nothing(x):
    pass


#creates trackbar UI
def filter_trackbar_create():
    print("trackback ready")
	# Create a black image
    img	= np.zeros((300,512,3), np.uint8) # creates a black window
    cv2.namedWindow('Webcam Heart Rate Monitor') #will open a window with the name in red
    cv2.createTrackbar('Amplification', 'Webcam Heart Rate Monitor', 75, 200, nothing)
    cv2.createTrackbar('Buffer Size', 'Webcam Heart Rate Monitor', 30, 200, nothing)

#grabs value from trackbar
def	filter_trackbar():
    alpha = cv2.getTrackbarPos('Amplification','Webcam Heart Rate Monitor')
    buffferSize = cv2.getTrackbarPos('Buffer Size','Webcam Heart Rate Monitor')
    return alpha, bufferSize


    
#Video Parameters
realWidth = 640
realHeight = 480
videoWidth = 320
videoHeight = 240
videoChannels = 3
videoFrameRate = 30

# Color Magnification Parameters
levels = 3
alpha = 100
minFrequency = 1.0
maxFrequency = 2.0
bufferSize = 150
bufferIndex = 0

# Output Display Parameters
font = cv2.FONT_HERSHEY_SIMPLEX
loadingTextLocation = (20, 100)
bpmTextLocation = (videoWidth//2 + 5, 100)
fontScale = 1
fontColor = (0,255,0)
lineType = 2
boxColor = (0, 255, 0)
boxWeight = 3

# Initialize Gaussian Pyramid
firstFrame = np.zeros((videoHeight, videoWidth, videoChannels))
firstGauss = buildGauss(firstFrame, levels+1)[levels]
videoGauss = np.zeros((bufferSize, firstGauss.shape[0], firstGauss.shape[1], videoChannels))
fourierTransformAvg = np.zeros((bufferSize))

# Bandpass Filter for Specified Frequencies
frequencies = (1.0*videoFrameRate) * np.arange(bufferSize) / (1.0*bufferSize)
mask = (frequencies >= minFrequency) & (frequencies <= maxFrequency)

# Heart Rate Calculation Variables
bpmCalculationFrequency = 15
bpmBufferIndex = 0
bpmBufferSize = 10
bpmBuffer = np.zeros((bpmBufferSize))

def main(bufferIndex, bpmBufferIndex):

    #initialize trackbar
    filter_trackbar_create()

    i = 0
    while (True):
        #reads frame from camera
        ret, frame = cap.read()

        if ret == True:
            
            #crop frame in for detection
            detectionFrame = frame[videoHeight//2:realHeight-videoHeight//2, videoWidth//2:realWidth-videoWidth//2, :]

            #grab values from trackbar
            alpha, bufferSize = filter_trackbar()
        
            # Construct Gaussian Pyramid
            videoGauss[bufferIndex] = buildGauss(detectionFrame, levels+1)[levels]
            fourierTransform = np.fft.fft(videoGauss, axis=0)
        
            # Bandpass Filter
            fourierTransform[mask == False] = 0
        
            # Grab a Pulse
            if bufferIndex % bpmCalculationFrequency == 0:
                i = i + 1
                for buf in range(bufferSize):
                    fourierTransformAvg[buf] = np.real(fourierTransform[buf]).mean()
                hz = frequencies[np.argmax(fourierTransformAvg)]
                bpm = 60.0 * hz
                bpmBuffer[bpmBufferIndex] = bpm
                bpmBufferIndex = (bpmBufferIndex + 1) % bpmBufferSize
        
            # Amplify
            filtered = np.real(np.fft.ifft(fourierTransform, axis=0))
            filtered = filtered * alpha
        
            # Reconstruct Resulting Frame
            filteredFrame = reconstructFrame(filtered, bufferIndex, levels)
            outputFrame = detectionFrame + filteredFrame
            outputFrame = cv2.convertScaleAbs(outputFrame)
        
            bufferIndex = (bufferIndex + 1) % bufferSize

            #apply box and text to frame
            frame[videoHeight//2:realHeight-videoHeight//2, videoWidth//2:realWidth-videoWidth//2, :] = outputFrame
            cv2.rectangle(frame, (videoWidth//2 , videoHeight//2), (realWidth-videoWidth//2, realHeight-videoHeight//2), boxColor, boxWeight)
            if i > bpmBufferSize:
                cv2.putText(frame, "BPM: %d" % bpmBuffer.mean(), bpmTextLocation, font, fontScale, fontColor, lineType)
            else:
                cv2.putText(frame, "Calculating BPM...", loadingTextLocation, font, fontScale, fontColor, lineType)
        
            if len(sys.argv) != 2:
                cv2.imshow("Webcam Heart Rate Monitor", frame)
        
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
        else:
            break

    cap.release()
    cv2.destroyAllWindows()

main(bufferIndex, bpmBufferIndex)

Video size:  640 x 480
trackback ready
